In [1]:
from dotenv import load_dotenv  
import pandas as pd
from woocommerce import API
import os

In [2]:
load_dotenv()
url = os.getenv("URL")
consumer_key = os.getenv("CONSUMER_KEY")
consumer_secret = os.getenv("CONSUMER_SECRET")


In [3]:
wcapi = API(
    url=url,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    wp_api=True,
    version="wc/v3"
)

In [4]:
datos = wcapi.get("orders?per_page=100").json()


In [5]:
# row = []

# for items in datos:

#     if items['status'] != "on-hold":
#         continue

#     id = items['id']
#     nombre_completo = items['billing']['first_name'] + " " + items['billing']['last_name']
#     address = items['billing']['address_1']
#     distrito = items['billing']['distrito']
#     provincia = items['billing']['provincia'] 
#     departamento = items['billing']['departamento']
#     monto = items['total']
#     phone = items['billing']['phone']

    
#     row.append([id,nombre_completo,address,distrito,provincia,departamento,monto,phone])








In [6]:
def payment_name(method_pay):
    if method_pay == 'woo-mercado-pago-basic':
        return 'Mercado Pago'
    if method_pay == 'micuentawebstd':
        return 'Pago Link - Izipay'
    else:
        return 'Revisar'
    

#'_billing_dni'
def get_meta_data(key,data):
    for meta_item in data['meta_data']:
        if meta_item['key'] == key:
            return meta_item['value']
    return None


def get_payment_id(method_pay,data):
    if method_pay == 'woo-mercado-pago-basic':
        return get_meta_data('_Mercado_Pago_Payment_IDs',data)
    if method_pay == 'micuentawebstd':
        return get_meta_data('Transaction ID',data)
    else:
        return 'Revisar'

In [7]:
row = []
    # order-shipped
    # if items['status'] != "on-hold":


for items in datos:

    if items['status'] != "processing":
        continue

    id = items['id']
    status = items['status']
    registro = items['date_created'].split('T')[0]
    nombre_completo = items['billing']['first_name'] + " " + items['billing']['last_name']
    address = items['billing']['address_1']
    distrito = items['billing']['distrito']
    provincia = items['billing']['provincia'] 
    departamento = items['billing']['departamento']
    monto = items['total']
    phone = items['billing']['phone'].replace(" ","")
    sku = items['line_items'][0]['sku']
    email = items['billing']['email']
    payment_method = payment_name(items['payment_method'])
    dni = get_meta_data('_billing_dni',items)
    payment_number = str(get_payment_id(items['payment_method'],items))
    
    row.append([id,status,registro,dni, nombre_completo,'-','-','-','-','-',address,'-',distrito,provincia,departamento,sku,monto,phone[-9:],'-','-','1',email,'-',payment_method,'Capturado',payment_number])

In [8]:
df = pd.DataFrame(row)
df = df.sort_values(by=0, ascending=True)
df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
3,9483,processing,2024-04-04,71656411,sara paucar rojas,-,-,-,-,-,...,2549.00,977749458,-,-,1,paucarrojassara@gmail.com,-,Pago Link - Izipay,Capturado,085249
2,9488,processing,2024-04-05,70069848,José Antonio,-,-,-,-,-,...,549.00,900027422,-,-,1,youngkrony2018@gmail.com,-,Pago Link - Izipay,Capturado,192141
1,9496,processing,2024-04-05,41763574,David menacho Huamancha,-,-,-,-,-,...,1549.00,945305839,-,-,1,dafacmd@hotmail.com,-,Pago Link - Izipay,Capturado,590008
0,9498,processing,2024-04-05,40836636,Enrique Freddy Huarote Salinas,-,-,-,-,-,...,649.00,961278141,-,-,1,huaroteenrique6@gmail.com,-,Pago Link - Izipay,Capturado,665141


In [9]:
if os.path.exists('reporte.csv'):
    os.remove('reporte.csv')
    df.to_csv('reporte.csv', sep=',', index=False, encoding='utf-8-sig')

else:
    df.to_csv('reporte.csv', sep=',', index=False, encoding='utf-8-sig')

    


In [11]:
one_product = wcapi.get("products").json()
print(one_product)

[{'id': 5704, 'name': 'iPhone 14 Pro', 'slug': 'iphone-14-64-128-256gb', 'permalink': 'https://catapu.com/comprar-iphones/iphone-14-64-128-256gb/', 'date_created': '2024-01-30T15:17:39', 'date_created_gmt': '2024-01-30T20:17:39', 'date_modified': '2024-04-05T16:27:31', 'date_modified_gmt': '2024-04-05T21:27:31', 'type': 'variable', 'status': 'publish', 'featured': False, 'catalog_visibility': 'visible', 'description': '<p class="salto"><strong>iPhone 14 Pro en Catapu: Innovación Avanzada y Rendimiento Profesional</strong></p>\n<p class="salto"><strong>Explora una Experiencia Visual Inigualable con la Pantalla ProMotion XDR</strong><br />\nSumérgete en la innovación visual con el iPhone 14 Pro de Catapu. Su impresionante pantalla ProMotion XDR redefine los estándares con colores precisos, contrastes dinámicos y una tasa de actualización elevada. Cada interacción se vuelve excepcional en esta pantalla que supera tus expectativas.</p>\n<p class="salto"><strong>Rendimiento de Élite con el 